### Factorization Machines:
### This notebook illustrates a distributed mplementation of Steffen Rendle's Factorization Machines on MPP Databases such as Greenplum and HAWQ.

### Reference: Factorization Machines By Steffen Rendle
### http://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf


### First create a database connection to Greenplum or HAWQ.

In [10]:
%run '../database_connectivity_setup.ipynb'

### Create a new schema for Factorization Machines

In [12]:
sql = """create schema ml;"""
psql.execute(sql,conn)
conn.commit()

### Two parallel gradient descent implementations are provided: 1) mini batch stochastic gradient descent (batch-sgd) and 2) regular batch gradient descent. The batch-sgd algorithm assumes the training data has been distributed randomly across multiple segments. The weights are updated per segment based on mini-batches of data in each segment and the weights are averaged in the end at the master node. The approach here is similar to Algorithm 3 in http://www.research.rutgers.edu/~lihong/pub/Zinkevich11Parallelized.pdf. The only difference being while the data per segment is randomly shuffled in each epoch for updating the parameters per mini-batch on that segment, the random distribution of the entire data across all segments happens only at the beginning.

### Let us first create the batch-sgd update function in PL/Python

In [3]:
sql = """drop function if exists ml.fm_sgd_update (
            text[], 
            float8[], 
            float8[], 
            int, 
            int, 
            float8, 
            float8, 
            float8, 
            float8,
            float8,
            float8,
            varchar
        );"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create or replace function ml.fm_sgd_update (
        xarr text[],
        yarr float8[],
        w float8[],
        n int,
        k int,
        miny float8,
        maxy float8,
        lambda0 float8,
        lambda1 float8,
        lambda2 float8,
        learning_rate float8,
        rseed varchar
    )
    returns float8[]
    AS
    $$
        import numpy as np
        
        def update_params_batch(
            x,
            y,
            w_t,
            n,
            k,
            miny,
            maxy,
            lambda0,
            lambda1,
            lambda2,
            learning_rate,
            b
        ):
            w0 = w_t[0]
            w  = np.reshape(np.array(w_t[1:n+1]), (1,n))
            v  = np.matrix(np.reshape(w_t[n+1:],(n,k)))

            y_hat = predict(x,w0,w,v)
            y_hat = np.minimum(maxy, y_hat)
            y_hat = np.maximum(miny, y_hat)
            term1 = 2*(y_hat - y[:,np.newaxis])
            gradw0 = np.sum(term1, axis = 0) + lambda0*w0
            if(gradw0.shape[0] <> 1 or gradw0.shape[1] <> 1):
                raise ValueError('Gradient w0 has more than 1 element')
            gradw = np.sum(np.multiply(term1,x), axis = 0) + lambda1*w
            if(gradw.shape[1] <> x.shape[1]):
                raise ValueError('Gradient w has incorrect number of elements')   
            gradv = np.zeros(v.shape)
            d = x.shape[0]
            instances = range(0,d)
            for dd in instances:
                xd = x[dd,:]
                xdv = xd*v
                gradv += term1[dd][0,0] * (
                            np.multiply(xd.T,np.repeat(xdv,xd.shape[1],axis=0)) -
                            np.multiply(v,np.square(xd.T))
                        ) 
            gradv += lambda2*v

            w0 -= (learning_rate/b*1.0)*gradw0
            w  -= (learning_rate/b*1.0)*gradw
            v  -= (learning_rate/b*1.0)*gradv
            
            return [w0[0,0]] + w[0,:].tolist() + np.array(np.reshape(v,(1,n*k))).tolist()[0]
            
        def update_params(
            x,
            y,
            w_t,
            n,
            k,
            miny,
            maxy,
            lambda0,
            lambda1,
            lambda2,
            learning_rate
        ):
            w0 = w_t[0]
            w  = np.reshape(np.array(w_t[1:n+1]),(1,n))
            v  = np.matrix(np.reshape(w_t[n+1:],(n,k)))

            y_hat = predict(x,w0,w,v)
            y_hat = np.minimum(maxy, y_hat)
            y_hat = np.maximum(miny, y_hat)
            term1 = 2*(y_hat - y)
            gradw0 = term1 + lambda0*w0
            if(gradw0.shape[0] <> 1 or gradw0.shape[1] <> 1):
                raise ValueError('Gradient w0 has more than 1 element')
            gradw = np.multiply(term1,x) + lambda1*w
            if(gradw.shape[1] <> x.shape[1]):
                raise ValueError('Gradient w has incorrect number of elements')   
            gradv = np.zeros(v.shape)
            xdv = x*v
            gradv += term1[0,0] * (
                        np.multiply(x.T,np.repeat(xdv,x.shape[1],axis=0)) -
                        np.multiply(v,np.square(x.T))
                    ) 
            gradv += lambda2*v
            w0 -= learning_rate*gradw0
            w  -= learning_rate*gradw
            v  -= learning_rate*gradv
            return [w0[0,0]] + w[0,:].tolist() + np.array(np.reshape(v,(1,n*k))).tolist()[0]
            
        def predict(x, w0, w, v):
            return (w0 + 
                x*w.T + 
                0.5*(np.sum(np.square(x*v) - (np.square(x)*np.square(v)),axis=1))
               )
        
        arr = ','.join(xarr)
        x = np.matrix(arr,dtype=float)
        x = x.reshape(x.shape[1]/n, n)
        d = x.shape[0]
        minibatch_size = 300
        indx = range(0,d)
        random_seed = None
        if (rseed != 'None'):
            random_seed = int('rseed')
        if (random_seed is not None):
            np.random.seed(random_seed)
        w_t = np.array(w)
        y = np.array(yarr,dtype=float)
        if (d >= 600):
            shuffledindx = np.random.permutation(indx)
            start = 0;
            end = minibatch_size
            while (start < d):
                xbatch = x[shuffledindx[start:end],:]
                ybatch = y[shuffledindx[start:end]]
                w_t = update_params_batch(
                    xbatch,
                    ybatch,
                    w_t,
                    n,
                    k,
                    miny,
                    maxy,
                    lambda0,
                    lambda1,
                    lambda2,
                    learning_rate,
                    xbatch.shape[0]
                )
                start += minibatch_size
                end += minibatch_size
                if (end > d):
                    end = d
        else:
            shuffledindx = np.random.permutation(indx)
            for i in shuffledindx:
                xd = x[i,:]
                yd = y[i]
                w_t = update_params(
                    xd,
                    yd,
                    w_t,
                    n,
                    k,
                    miny,
                    maxy,
                    lambda0,
                    lambda1,
                    lambda2,
                    learning_rate
                )
        return w_t
    $$language plpythonu;
"""
psql.execute(sql,conn)
conn.commit()

### Let us next create the regular batch gradient function in PL/Python. Here, we compute only the gradient of the loss function in a parallel fashion. The gradients are added up in the master node along with the gradient of the regularization component and the weights are updated in the master

In [8]:
sql = """drop function if exists ml.fm_loss_gradient(
            text[], 
            float8[], 
            float8[], 
            int, 
            int, 
            float8, 
            float8,
            float8
        );"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create or replace function ml.fm_loss_gradient (
        xarr text[],
        yarr float8[],
        w float8[],
        n int,
        k int,
        miny float8,
        maxy float8,
        trnsize float8
    )
    returns float8[]
    AS
    $$
        import numpy as np
        
        def compute_loss_gradient(
            x,
            y,
            w_t,
            n,
            k,
            miny,
            maxy,
            trnsize
        ):
            d = x.shape[0]
            if (d == 1):
                loss_grad = compute_loss_gradient_single(x,y,w_t,n,k,miny,maxy)
                return loss_grad
                
            w0 = w_t[0]
            w  = np.reshape(np.array(w_t[1:n+1]),(1,n))
            v  = np.matrix(np.reshape(w_t[n+1:],(n,k)))

            y_hat = predict(x,w0,w,v)
            y_hat = np.minimum(maxy, y_hat)
            y_hat = np.maximum(miny, y_hat)
            term1 = (1.0/trnsize)*(y_hat - y[:,np.newaxis])
            gradw0 = np.sum(term1, axis = 0)
            if(gradw0.shape[0] <> 1 or gradw0.shape[1] <> 1):
                raise ValueError('Gradient w0 has more than 1 element')
            gradw = np.array(np.sum(np.multiply(term1,x), axis = 0))
            if(gradw.shape[1] <> x.shape[1]):
                raise ValueError('Gradient w has incorrect number of elements')   
            gradv = np.matrix(np.zeros(v.shape))
            instances = range(0,d)
            for dd in instances:
                xd = x[dd,:]
                xdv = xd*v
                gradv += term1[dd][0,0] * (
                            np.multiply(xd.T,np.repeat(xdv,xd.shape[1],axis=0)) -
                            np.multiply(v,np.square(xd.T))
                        ) 

            loss_grad = [gradw0[0,0]] + gradw[0,:].tolist() + np.array(np.reshape(gradv,(1,n*k))).tolist()[0]
            return loss_grad
            
        def compute_loss_gradient_single(
            x,
            y,
            w_t,
            n,
            k,
            miny,
            maxy,
            trnsize
        ):
            w0 = w_t[0]
            w  = np.reshape(np.array(w_t[1:n+1]),(1,n))
            v  = np.matrix(np.reshape(w_t[n+1:],(n,k)))
            
            y_hat = predict(x,w0,w,v)
            y_hat = np.minimum(maxy, y_hat)
            y_hat = np.maximum(miny, y_hat)
            term1 = (1.0/trnsize)*(y_hat - y)
            gradw0 = term1
            if(gradw0.shape[0] <> 1 or gradw0.shape[1] <> 1):
                raise ValueError('Gradient w0 has more than 1 element')
            gradw = np.array(np.multiply(term1,x))
            if(gradw.shape[1] <> x.shape[1]):
                raise ValueError('Gradient w has incorrect number of elements')   
            gradv = np.matrix(np.zeros(v.shape))
            xdv = x*v
            gradv += term1[0,0] * (
                        np.multiply(x.T,np.repeat(xdv,x.shape[1],axis=0)) -
                        np.multiply(v,np.square(x.T))
                    ) 
            loss_grad = [gradw0[0,0]] + gradw[0,:].tolist() + np.array(np.reshape(gradv,(1,n*k))).tolist()[0]
            return loss_grad
            
        def predict(x, w0, w, v):
            return (w0 + 
                x*w.T + 
                0.5*(np.sum(np.square(x*v) - (np.square(x)*np.square(v)),axis=1))
               )
        
        arr = ','.join(xarr)
        x = np.matrix(arr,dtype=float)
        x = x.reshape(x.shape[1]/n, n)
        w_t = np.array(w,dtype=float)
        y = np.array(yarr,dtype=float)
        loss_grad = compute_loss_gradient(x,y,w_t,n,k,miny,maxy,trnsize)
        return loss_grad
    $$language plpythonu;
"""
psql.execute(sql,conn)
conn.commit()

### We will now create the driver function for calling the per-segment batch or batch-sgd PL/Python UDFs. This will be the function that the end user will call

In [9]:
# Driver function
sql = """drop function if exists ml.fm_fit(varchar,varchar,varchar,varchar,varchar,varchar);"""
psql.execute(sql,conn)
conn.commit()

# Parameters of the fm_fit function include:
# 1. source_table - table containing the feature vectors and dependent variable as two separate columns
# 2. id_col_name - an id column for each row
# 3. x_col_name - features
# 4. y_col_name - dependent variable
# 5. parameters - learning parameters: regularizers (lambda0, lambda1, lambda2), number of iterations (n_iter), learning rate
# (learning_rate), factorization hyper-parameter (k), random seed (integer for batch-sgd), optim (batch or batch-sgd)
# out_table - name of the table where the model will be stored

sql = """
    create or replace function ml.fm_fit (
        src_table varchar,
        id_col_name varchar,
        x_col_name varchar,
        y_col_name varchar,
        parameters varchar,
        out_table varchar
    )
    returns text
    AS
    $$
        import numpy as np
        
        # Initialize the parameters to their default values
        lambda0 = 1
        lambda1 = 1
        lambda2 = 1
        n_iter = 100
        learning_rate = 1
        k = 50
        random_seed = 'None'
        optim = 'batch-sgd'
        params = {}

        for item in parameters.split(','):
            params[item.split('=')[0].strip()] = item.split('=')[1].strip()
        
        if params.has_key('lambda0'):
            lambda0 = float(params['lambda0'])
        if params.has_key('lambda1'):
            lambda1 = float(params['lambda1'])
        if params.has_key('lambda2'):
            lambda2 = float(params['lambda2'])
        if params.has_key('n_iter'):
            n_iter = int(params['n_iter'])
        if params.has_key('learning_rate'):
            learning_rate = float(params['learning_rate'])
        if params.has_key('k'):
            k = int(params['k'])
        if params.has_key('random_seed'):
            random_seed = int(params['random_seed'])
        if params.has_key('optim'):
            optim = params['optim']
            plpy.info('optimization selected: ' + optim)
        
        sql = '''
            select array_upper({x_col_name},1) as n from {src_table} limit 1;
        '''.format(x_col_name=x_col_name,src_table=src_table)
        rv = plpy.execute(sql)
        n = rv[0]['n']
        
        sql = '''
            select count(distinct {id_col_name}) as trnsize from {src_table} limit 1;
        '''.format(id_col_name=id_col_name,src_table=src_table)
        rv = plpy.execute(sql)
        trnsize = rv[0]['trnsize']
        
        sql = '''
            select min({y_col_name})::float8 as miny, max({y_col_name})::float8 as maxy from {src_table} limit 1;
        '''.format(y_col_name=y_col_name,src_table=src_table)
        rv = plpy.execute(sql)
        miny = rv[0]['miny']
        maxy = rv[0]['maxy']
        
        w0 = 0
        w = np.zeros([1,n])
        v = np.random.normal(scale=0.1,size=(n,k))
        
        w_t = [w0] + w[0,:].tolist() + np.array(np.reshape(v,(1,n*k))).tolist()[0]
        
        for i in range(0,n_iter):
            plpy.info("Epoch number: " + str(i))
            
            if optim == 'batch':
                sql = \"""
                    select
                        ml.fm_loss_gradient (
                            xarr,
                            yarr,
                            array{w_t},
                            {n},
                            {k},
                            {miny},
                            {maxy},
                            {trnsize}
                        ) as loss_grad
                    from (
                        select
                            gp_segment_id,
                            array_agg(array_to_string({x_col_name},',') order by {id_col_name}) as xarr,
                            array_agg({y_col_name} order by {id_col_name}) as yarr
                        from            
                            {src_table}
                        group by 1
                    )q1;
                \""".format(
                        src_table=src_table,
                        x_col_name=x_col_name,
                        y_col_name=y_col_name,
                        id_col_name=id_col_name,
                        w_t=w_t,
                        n=n,
                        k=k,
                        miny=miny,
                        maxy=maxy,
                        trnsize=trnsize
                    )
                rv = plpy.execute(sql)
                nrows = len(rv)
                loss_grad = np.zeros(shape=(1,n+1+(n*k)))
                for r in range(0,nrows):
                    loss_grad += np.array(rv[r]['loss_grad'])

                loss_grad = loss_grad[0]
                loss_grad_w0 = loss_grad[0]
                loss_grad_w  = np.reshape(np.array(loss_grad[1:n+1]),(1,n))
                loss_grad_v  = np.matrix(np.reshape(loss_grad[n+1:],(n,k)))
                gradw0 = loss_grad_w0 + lambda0*w0
                gradw  = loss_grad_w + lambda1*w
                gradv  = loss_grad_v + lambda2*v
                w0 -= (learning_rate)*gradw0
                w  -= (learning_rate)*gradw
                v  -= (learning_rate)*gradv
                w_t = [w0] + w[0,:].tolist() + np.array(np.reshape(v,(1,n*k))).tolist()[0]
            elif optim == 'batch-sgd':
                sql = \"""
                    select
                        ml.fm_sgd_update (
                            xarr,
                            yarr,
                            array{w_t},
                            {n},
                            {k},
                            {miny},
                            {maxy},
                            {lambda0},
                            {lambda1},
                            {lambda2},
                            {learning_rate},
                            '{random_seed}'
                        ) as w_curr
                    from (
                        select
                            gp_segment_id,
                            array_agg(array_to_string({x_col_name},',') order by {id_col_name}) as xarr,
                            array_agg({y_col_name} order by {id_col_name}) as yarr
                        from            
                            {src_table}
                        group by 1
                    )q1;
                \""".format(
                        src_table=src_table,
                        x_col_name=x_col_name,
                        y_col_name=y_col_name,
                        id_col_name=id_col_name,
                        w_t=w_t,
                        n=n,
                        k=k,
                        miny=miny,
                        maxy=maxy,
                        lambda0=lambda0,
                        lambda1=lambda1,
                        lambda2=lambda2,
                        learning_rate=learning_rate,
                        random_seed=random_seed
                    )
                rv = plpy.execute(sql)
                nrows = len(rv)
                w_t = np.zeros(shape=(1,n+1+(n*k)))
                for r in range(0,nrows):
                    w_t += np.array(rv[r]['w_curr'])
                w_t[0,:] = w_t[0,:]/nrows
                w_t = w_t[0,:].tolist()
            
        sql = \"""
            create table {out_table} (
                n int4,
                k int4,
                miny float8,
                maxy float8,
                w float8[]
            ) distributed randomly;
        \""".format(out_table=out_table)
        plpy.execute(sql)
        
        sql = \"""
            insert into {out_table} values ({n},{k},{miny},{maxy},array{w});
        \""".format(out_table=out_table,n=n,k=k,miny=miny,maxy=maxy,w=w_t)
        plpy.execute(sql)
        
        return "Table {out_table} created.".format(out_table=out_table)
    $$language plpythonu;
"""
psql.execute(sql,conn)
conn.commit()

### Let us next define the prediction function

In [6]:
sql = """
    create or replace function ml.fm_predict (
        xarr float8[],
        n int,
        k int,
        w_t float8[],
        miny float8,
        maxy float8
    )
    returns float8
    AS
    $$
        import numpy as np
        x = np.matrix(xarr, dtype=float)
        w0 = w_t[0]
        w  = np.reshape(np.array(w_t[1:n+1]),(1,n))
        v  = np.matrix(np.reshape(w_t[n+1:],(n,k)))
        y_hat = (w0 + 
                x*w.T + 
                0.5*(np.sum(np.square(x*v) - (np.square(x)*np.square(v)),axis=1))
               )

        y_hat = np.minimum(maxy, y_hat)
        y_hat = np.maximum(miny, y_hat)
        return y_hat[0,0]       
    $$language plpythonu;
"""
psql.execute(sql,conn)
conn.commit()

### Experiment with MovieLens 100-K Ratings Data. The dataset is a popular dataset for recommender systems and can be downloaded here: http://grouplens.org/datasets/movielens/100k/

### We will load the data into tables in GPDB and HAWQ


In [8]:
sql = """drop table if exists ml.fm_train_data;"""
psql.execute(sql,conn)
conn.commit()

sql = """drop table if exists ml.fm_test_data;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table ml.fm_train_data (
        user_id int4,
        item_id int4,
        rating int4,
        ts double precision
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table ml.fm_test_data (
        user_id int4,
        item_id int4,
        rating int4,
        ts double precision
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()
#COPY ml.fm_train_data FROM '/home/gmurlidhar/data/movielens/ua.base' CSV DELIMITER E'\t' NULL '';
#COPY ml.fm_test_data FROM '/home/gmurlidhar/data/movielens/ua.test' CSV DELIMITER E'\t' NULL '';

### PL/Python UDF to create a feature vector from the [user_id, item_id] tuples

In [18]:
sql = """drop function if exists ml.create_fm_feat_vec(int4,int4,int4[],int4[]) cascade;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    CREATE OR REPLACE FUNCTION ml.create_fm_feat_vec(
        user_id int4,
        item_id int4,
        all_users int4[],
        all_items int4[]
    )
    RETURNS int4[]
    AS
    $$
        import numpy as np
        useridx = {}
        itemidx = {}
        numusers = len(all_users)
        numitems = len(all_items)
        if 'useridx' not in SD:
            for i in range(0,numusers):
                useridx[all_users[i]] = i
            SD['useridx'] = useridx
        else:
            useridx = SD['useridx']
        
        if 'itemidx' not in SD:
            for i in range(0,numitems):
                itemidx[all_items[i]] = i
            SD['itemidx'] = itemidx
        else:
            itemidx = SD['itemidx']
            
        x = [0 for i in range(0,numusers+numitems)]
        
        if useridx.has_key(user_id):
            x[useridx[user_id]] = 1
        if itemidx.has_key(item_id):
            x[numusers+itemidx[item_id]] = 1
        return x
    $$ LANGUAGE PLPYTHONU;
"""
psql.execute(sql,conn)
conn.commit()

### Create the training table

In [15]:
sql = """drop table if exists ml.fm_train_table;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table ml.fm_train_table as (
        select
            row_number() over() as id,
            ml.create_fm_feat_vec (
                t1.user_id,
                t1.item_id,
                t2.all_users::int4[],
                t3.all_items::int4[]
            ) as x,
            t1.rating as y
        from
            ml.fm_train_data t1,
            (
                select
                    array_agg(user_id order by user_id) as all_users
                from (
                    select
                        user_id
                    from
                        ml.fm_train_data
                    group by 1
                )t21
            ) t2,
            (
                select
                    array_agg(item_id order by item_id) as all_items
                from (
                    select
                        item_id
                    from
                        ml.fm_train_data
                    group by 1
                )t31
            ) t3
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

In [7]:
sql = """select count(*) as num_rows from ml.fm_train_table;"""
print psql.read_sql(sql,conn)

sql = """select array_upper(x,1) as num_cols from ml.fm_train_table group by 1;"""
print psql.read_sql(sql,conn)

   num_rows
0     90570
   num_cols
0      2623


### Call the fm_fit function

In [ ]:
sql = """
    select 
        ml.fm_fit (
            'ml.fm_train_table',
            'id',
            'x',
            'y',
            'lambda0 = 0.01, lambda1 = 0.01, lambda2 = 0.01, k = 10, n_iter = 100, learning_rate = 0.5, optim=batch-sgd',
            'ml.fm_mdl_table'
        ); 
"""
df = psql.read_sql(sql,conn)

### Create the testing table to test the model

In [19]:
sql = """drop table if exists ml.fm_test_table;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table ml.fm_test_table as (
        select
            row_number() over() as id,
            ml.create_fm_feat_vec (
                t1.user_id,
                t1.item_id,
                t2.all_users::int4[],
                t3.all_items::int4[]
            )::float8[] as x,
            t1.rating::float8 as y
        from
            ml.fm_test_data t1,
            (
                select
                    array_agg(user_id order by user_id) as all_users
                from (
                    select
                        user_id
                    from
                        ml.fm_train_data
                    group by 1
                )t21
            ) t2,
            (
                select
                    array_agg(item_id order by item_id) as all_items
                from (
                    select
                        item_id
                    from
                        ml.fm_train_data
                    group by 1
                )t31
            ) t3
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

In [11]:
sql = """select count(*) as num_rows from ml.fm_test_table;"""
print psql.read_sql(sql,conn)

sql = """select array_upper(x,1) as num_cols from ml.fm_test_table group by 1;"""
print psql.read_sql(sql,conn)

   num_rows
0      9430
   num_cols
0      2623


### Compute predictions on the test set

In [12]:
sql = """drop table if exists ml.fm_test_predictions;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table ml.fm_test_predictions as (
        select
            t1.id,
            t1.y,
            ml.fm_predict (
                t1.x,
                t2.n,
                t2.k,
                t2.w,
                t2.miny,
                t2.maxy
            ) as y_hat
        from
            ml.fm_test_table t1,
            ml.fm_mdl_table t2 
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

### Compute MSE!

In [13]:
sql = """
    select
        avg(se) as mean_squared_error
    from (
        select
            (y_hat-y)^2 as se
        from
            ml.fm_test_predictions
    )t
"""
psql.read_sql(sql,conn)

,mean_squared_error
0,1.062072


In [14]:
conn.close()